In [ ]:
import beamtest_analysis_helper as helper
from glob import glob
from pathlib import Path
from datetime import datetime, timedelta
import pandas
import re
from natsort import natsorted
import copy

In [ ]:
chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_EPIR_Pair1", "ET2_BAR_4", "ET2_BAR_6", "ET2_EPIR_Pair4"]
# chip_figtitles = ['ET2 EPIR FBK UFSD4 W10 5-5']  # Pair 3
#chip_figtitles = ['ET2 EPIR FBK UFSD4 W9 3-5']  # Pair 2
# chip_figtitles = ['ET2 EPIR FBK UFSD4 W13 4-5']  # CERN Trigger
# chip_figtitles = ['ET2 Barcelona FBK UFSD4 W12 2-3']  # W12 2-3
#chip_figtitles = ['ET2 Barcelona FBK UFSD4 W12 5-3']  # W12 5-3
#chip_figtitles = ['ET2 Barcelona FBK UFSD4 W15 3-4']  # W15 3-4
#chip_figtitles = ['ET2 EPIR FBK Pair 4']  # Pair 4
#chip_figtitles = ['ET2 Barcelona FBK UFSD4 W12 5-5']  # W12 5-5
chip_figtitles = chip_names
chip_fignames = chip_figtitles
fig_path = './'

In [ ]:
# files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/Pair2/2023-10-31_Array_Test_Results_Overnight/*/*.dat')
# files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/CERNTrigger/2023-11-12_Array_Test_Results_Overnight/*/*.dat')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/Pair3/2023-11-10_Array_Test_Results_Weekend/*/*.dat')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/W12_2_3/2023-11-18_Array_Test_Results_Weekend/*/*.dat')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/W12_5_3/2023-11-20_Array_Test_Results/*/*.dat')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/W15_3_4/2023-11-21_Array_Test_Results_Overnight/*/*.dat')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/Pair4/2023-11-27_Array_Test_Results_Overnight/*/*.nem')
#files = glob('/run/media/daq/T7/BumpBondedCosmicRuns/W12_5_5/2023-11-28_Array_Test_Results_1hour/*/*.nem')
#files = glob('/run/media/daq/T7/2023-11-01_Array_Test_Results/*/*.dat')
# files = glob('../../ETROC-Data/2023-12-05_Array_Test_Results/SelfTrigger_ALL_OR_Readout_b0_EPIR_Pair1_b1_BAR_4_b2_BAR_6_b3_EPIR_Pair4_settings_offset_12_loop_0/*.nem')
#files = glob('/run/media/daq/T7/2023-12-05_Array_Test_Results/beamprep_SelfTrigger_EPIR_Pair1_Readout_b0_EPIR_Pair1_b1_BAR_4_b2_BAR_6_b3_EPIR_Pair4_settings_offset_15_loop_0/*.nem')
#files = glob('/run/media/daq/T7/Run_CNM_Alignment/loop_0/*nem')
files = glob('/run/media/daq/T7/Run_PACTECH234_02_22_OVERNIGHT_OFFSET5/loop_*/*nem')
#files = natsorted(files)

#files = files[30:90]

print(files)

# df = helper.toSingleDataFrame(files=files)
df = helper.toSingleDataFrame_newEventModel(files=files)
#helper.toSingleDataFramePerDirectory('/run/media/daq/T7/2023-10-28_Array_Test_Results', 'SelfTrigger_ET2_EPIR_Pair3_HV300_Readout_offset10_lowpower_cosmics_loop_*')

In [ ]:
df = pandas.read_feather('./cosmic_run_Feb22.feather')

# Inclusive

In [ ]:
helper.plot_occupany_map(df, chip_labels, chip_figtitles, "inclusive")

In [ ]:
helper.plot_TDC_summary_table(df, chip_labels, "cal")

In [ ]:
helper.plot_TDC_summary_table(df, chip_labels, "toa")

In [ ]:
helper.plot_TDC_summary_table(df, chip_labels, "tot")

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels)
helper.plot_1d_TDC_histograms(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], 
                                fig_path, save=False, tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], 
                                fig_path, save=False, tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], 
                                fig_path, save=False, tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_inclusive, chip_names[3], chip_fignames[3], chip_figtitles[3], 
                                fig_path, save=False, tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

# Event Based Selection

In [ ]:
df_event_selection = df.copy()

selection = (df_event_selection['board'] == 0) & (df_event_selection['row'] == 13) & (df_event_selection['col'] == 4)

selected_events = df_event_selection.loc[selection]['evt']

print(selected_events)

df_event_selection = df_event_selection.loc[df_event_selection['evt'].isin(selected_events)]
df_event_selection = df_event_selection.loc[~((df_event_selection['board'] == 0) & (df_event_selection['row'] == 13) & (df_event_selection['col'] == 4))]

In [ ]:
helper.plot_occupany_map(df_event_selection, chip_labels, chip_figtitles, "event selected")


In [ ]:
h_event_selected = helper.return_hist(df_event_selection, chip_names, chip_labels)
helper.plot_1d_TDC_histograms(h_event_selected, chip_names[3], chip_fignames[3], chip_figtitles[3], 
                                fig_path, save=False, tag="event_selected", fig_tag=", event selected", slide_friendly=True)

# Plot selected pixels

Useful for plotting a single noisy pixel for instance, or a group of pixels

In [ ]:
selected_pixels = [
#    (10, 8),
#    (12, 14),
#    (3, 10),
#    (2, 10),
#    (4, 12),
#    (5, 13),
#    (3, 11),
    (13, 4),
#    (15, 12),
#    (12, 14),
#    (13, 14),
#    (2, 15),
]

print("Number of Selected Pixels:", len(selected_pixels))

df_selected = df.copy()

for pixel in selected_pixels:
    df_selected = df_selected.loc[((df_selected['row'] == pixel[0]) & (df_selected['col'] == pixel[1]))].copy()

In [ ]:
helper.plot_occupany_map(df_selected, chip_labels, chip_figtitles, "selected")

In [ ]:
helper.plot_TDC_summary_table(df_selected, chip_labels, "cal")

In [ ]:
helper.plot_TDC_summary_table(df_selected, chip_labels, "toa")

In [ ]:
helper.plot_TDC_summary_table(df_selected, chip_labels, "tot")

In [ ]:
h_selected = helper.return_hist(df_selected, chip_names, chip_labels)
helper.plot_1d_TDC_histograms(h_selected, chip_names[0], chip_fignames[0], chip_figtitles[0], 
                                fig_path, save=False, tag="selected", fig_tag=", selected", slide_friendly=True)

# Mask selected pixels

Remove the selected pixels from processing

In [ ]:
masked_pixels = [
#    (10, 8),
#    (12, 14),
#    (3, 10),
#    (2, 10),
#    (4, 12),
#    (3, 11),
#    (15, 11),
#    (5, 13),
#    (15, 12),
#    (12, 14),
#    (14, 15),
#    (15, 13),
#    (15, 14),
    (6, 1),
    #(2, 15),
#    (13, 14),
#    (2, 15),
]

print("Number of Masked Pixels:", len(masked_pixels))

df_masked = df.copy()

for pixel in masked_pixels:
    df_masked = df_masked.loc[~((df_masked['row'] == pixel[0]) & (df_masked['col'] == pixel[1]))].copy()

print("Number of Events:",len(df_masked))


In [ ]:
helper.plot_occupany_map(df_masked, chip_labels, chip_figtitles, "masked")

In [ ]:
helper.plot_TDC_summary_table(df_masked, chip_labels, "cal")

In [ ]:
helper.plot_TDC_summary_table(df_masked, chip_labels, "toa")

In [ ]:
helper.plot_TDC_summary_table(df_masked, chip_labels, "tot")

In [ ]:
h_masked = helper.return_hist(df_masked, chip_names, chip_labels)
helper.plot_1d_TDC_histograms(h_masked, chip_names[0], chip_fignames[0], chip_figtitles[0], 
                                fig_path, save=False, tag="masked", fig_tag=", masked", slide_friendly=True)

# Filter Event

For applying cuts to the TOT, TOA or CAL

In [ ]:
df_filtered = df.copy()

#df_filtered = df_filtered.loc[(df_filtered['tot'] < 100) & (df_filtered['tot'] < 10000)].copy()
#df_filtered = df_filtered.loc[(df_filtered['tot'] < 100) | (df_filtered['board'] != 3)].copy()
df_filtered = df_filtered.loc[(df_filtered['tot'] > 100) & (df_filtered['tot'] < 10000)].copy()
#df_filtered = df_filtered.loc[(df_filtered['cal'] > 194) & (df_filtered['tot'] < 10000)].copy()
#df_filtered = df_filtered.loc[(df_filtered['cal'] < 200) & (df_filtered['tot'] < 10000)].copy()
#df_filtered = df_filtered.loc[(df_filtered['cal'] > 194) & (df_filtered['cal'] < 200)].copy()

pixels = [
#    (5, 13),
#    (3, 11),
#    (15, 11),
]

for pixel in pixels:
    df_filtered = df_filtered.loc[~((df_filtered['row'] == pixel[0]) & (df_filtered['col'] == pixel[1]))].copy()

In [ ]:
helper.plot_occupany_map(df_filtered, chip_labels, chip_figtitles, "filtered")

In [ ]:
helper.plot_TDC_summary_table(df_filtered, chip_labels, "cal")

In [ ]:
helper.plot_TDC_summary_table(df_filtered, chip_labels, "toa")

In [ ]:
helper.plot_TDC_summary_table(df_filtered, chip_labels, "tot")

In [ ]:
h_filtered = helper.return_hist(df_filtered, chip_names, chip_labels)
helper.plot_1d_TDC_histograms(h_filtered, chip_names[3], chip_fignames[3], chip_figtitles[3], 
                                fig_path, save=False, tag="masked", fig_tag=", masked", slide_friendly=True)

# Save Data

In [ ]:
#df.to_csv('ET2_EPIR_FBK_Pair4_single_cosmic.csv', index=False)
df.to_feather('test.feather')